<a href="https://colab.research.google.com/github/savinyu/VLG_Project/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.utils import compute_class_weight


train_df = pd.read_csv('train.csv')

X = train_df.drop('labels', axis=1).values
Y = train_df['labels'].values


X = X.reshape(X.shape[0], X.shape[1], 1)
X = X / 4



X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1200, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, strides=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.45))

model.add(Dense(1, activation='sigmoid'))

# tf.keras.applications.ResNet50(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,

# )
# tf.keras.applications.VGG19(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )



model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=10, class_weight=class_weights_dict)

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)


test_df = pd.read_csv('test.csv')


test = test_df.drop('id', axis=1).values
test = test.reshape(test.shape[0], test.shape[1], 1)


pred_probs = model.predict(test)
predicted_labels = (pred_probs > 0.5).astype(int)
predicted_labels = predicted_labels.reshape(predicted_labels.shape[0])


output_data = []
for i, value in enumerate(predicted_labels):
    output_data.append([i + 1, value])

output_file = 'output.csv'

with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['id', 'label'])
    writer.writerows(output_data)



print("CSV Output Done")


Epoch 1/10
26/26 [==============================] - 2s 10ms/step - loss: 0.6857 - accuracy: 0.5609
Epoch 2/10
26/26 [==============================] - 0s 9ms/step - loss: 0.6001 - accuracy: 0.6698
Epoch 3/10
26/26 [==============================] - 0s 8ms/step - loss: 0.5047 - accuracy: 0.7247
Epoch 4/10
26/26 [==============================] - 0s 8ms/step - loss: 0.4451 - accuracy: 0.7702
Epoch 5/10
26/26 [==============================] - 0s 9ms/step - loss: 0.4132 - accuracy: 0.8046
Epoch 6/10
26/26 [==============================] - 0s 8ms/step - loss: 0.3698 - accuracy: 0.8157
Epoch 7/10
26/26 [==============================] - 0s 8ms/step - loss: 0.3259 - accuracy: 0.8504
Epoch 8/10
26/26 [==============================] - 0s 9ms/step - loss: 0.2986 - accuracy: 0.8697
Epoch 9/10
26/26 [==============================] - 0s 8ms/step - loss: 0.2436 - accuracy: 0.8935
Epoch 10/10
26/26 [==============================] - 0s 2ms/step
F1 score: 0.6143497757847534
71/71 [================

In [165]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras import datasets, models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


df = pd.read_csv('train.csv')


x = df.drop('labels', axis=1)
y = df['labels']

X = x.values
X = X.reshape(x.shape[0], x.shape[1], 1)
X = X/4


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])


oversampler = RandomOverSampler(random_state=42)

X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)



X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0], X_train.shape[1], 1)

# Model
# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1200, 1)))
# model.add(MaxPooling1D(pool_size=2))

# model.add(Conv1D(filters=64, kernel_size=3, strides=2, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(rate=0.45))

# model.add(Dense(128, activation='relu'))
# model.add(Dropout(rate=0.4))

# model.add(Dense(1, activation='sigmoid'))

#Another Model Self (2D)

# model = models.Sequential([
#     layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=( 1200 , 1)),
#     layers.MaxPooling2D((2, 2)),

#     layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),

#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(10, activation='softmax')
# ])
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,

)
# tf.keras.applications.VGG19(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train_resampled, y_train_resampled, batch_size=128, epochs=20)


y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)


f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


df_test = pd.read_csv('test.csv')

test = df_test.drop('id', axis=1).values

test = test.reshape(test.shape[0], test.shape[1], 1)


pred_probs = model.predict(test)

predicted_labels = (pred_probs > 0.5).astype(int)

predicted_labels = predicted_labels.reshape(-1)



data = []
for i, value in enumerate(predicted_labels):
    data.append([i + 1, value])



csv_file = 'output.csv'



with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'label'])
    writer.writerows(data)



print("CSV file created successfully.")


Epoch 1/20
39/39 [==============================] - 2s 9ms/step - loss: 0.1212 - accuracy: 0.9565
Epoch 2/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0672 - accuracy: 0.9799
Epoch 3/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0618 - accuracy: 0.9819
Epoch 4/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0378 - accuracy: 0.9883
Epoch 5/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0344 - accuracy: 0.9897
Epoch 6/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0268 - accuracy: 0.9908
Epoch 7/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0531 - accuracy: 0.9801
Epoch 8/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0386 - accuracy: 0.9856
Epoch 9/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0340 - accuracy: 0.9858
Epoch 10/20
39/39 [==============================] - 0s 8ms/step - loss: 0.0236 - accuracy: 0.9928
Epoch 11/20
39/39 [

Using Conv2D
